# Training notebook
Used to replicate training procedures. Based on the code of this repo: https://github.com/hrsht-13/Breast-Cancer-Detection
You will need the phantom images in the corresponding folder

In [1]:
# Imports
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from keras_preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
import albumentations as A
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
import cv2
import os
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout,AveragePooling2D
from tensorflow.keras.applications.resnet import ResNet50
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3,DenseNet201,EfficientNetB7, MobileNetV2,Xception,VGG16,NASNetMobile
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Model
from keras.models import Sequential
from keras.regularizers import *
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

In [3]:
def preprocess(image):
    kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
    im = cv2.filter2D(image, -1, kernel)
    #out2 = skimage.exposure.rescale_intensity(im, in_range=(150,200), out_range=(0,255))
    out1=(cv2.normalize(im, (224,224),0, 255, cv2.NORM_MINMAX))
    return out1

In [4]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [5]:
# Loading the dataset and splitting it into train and validation
train=pd.read_csv("content/phantom_images/Training_set.csv")
X_train, X_valid = train_test_split(train, test_size=0.1,stratify=train["stratlabel"],shuffle=True, random_state = 42)

len of training set 4304
len of training set 3873
len of validation set 431


In [6]:
# Train generator
datagen=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess)
train_generator=datagen.flow_from_dataframe(
dataframe=X_train,
directory="content/phantom_images/train",
x_col="filename",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

Found 3873 validated image filenames belonging to 3 classes.


In [7]:
# Validation generator
valid_datagen=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess)
valid_generator=valid_datagen.flow_from_dataframe( 
dataframe=X_valid,
directory="content/phantom_images/train",
x_col="filename",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

Found 431 validated image filenames belonging to 3 classes.


## Experiment1 1:
Only last 3 neurons

In [27]:
checkpoint_path = "experiment1_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
for layer in conv_base.layers:
        layer.trainable = False
model.layers[-1].trainable = True
model.layers[-3].trainable = False
model.layers[-5].trainable = False
model.layers[-7].trainable = False

/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 1222 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [28]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

Epoch 1/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3657 - f1_score: 0.3041 - loss: 1.5959 - precision: 0.3561 - recall: 0.2655

/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: saving model to experiment1_ckpts/training_1.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 153s 1s/step - accuracy: 0.3655 - f1_score: 0.3040 - loss: 1.5961 - precision: 0.3560 - recall: 0.2653 - val_accuracy: 0.5174 - val_f1_score: 0.0000e+00 - val_loss: 1.0608 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3524 - f1_score: 0.2936 - loss: 1.5966 - precision: 0.3466 - recall: 0.2546
Epoch 2: saving model to experiment1_ckpts/training_2.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.3523 - f1_score: 0.2936 - loss: 1.5967 - precision: 0.3466 - recall: 0.2546 - val_accuracy: 0.4339 - val_f1_score: 0.0821 - val_loss: 1.1135 - val_precision: 0.3571 - val_recall: 0.0464
Epoch 3/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3461 - f1_score: 0.2907 - loss: 1.6015 - precision: 0.3400 - recall: 0.2539
Epoch 3: saving model to experiment1_ckpts/training_3.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━

Epoch 33/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3904 - f1_score: 0.3035 - loss: 1.3235 - precision: 0.3851 - recall: 0.2505
Epoch 33: saving model to experiment1_ckpts/training_33.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.3903 - f1_score: 0.3035 - loss: 1.3235 - precision: 0.3851 - recall: 0.2504 - val_accuracy: 0.4014 - val_f1_score: 0.2853 - val_loss: 1.3181 - val_precision: 0.3764 - val_recall: 0.2297
Epoch 34/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3799 - f1_score: 0.2819 - loss: 1.3446 - precision: 0.3619 - recall: 0.2309
Epoch 34: saving model to experiment1_ckpts/training_34.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.3800 - f1_score: 0.2820 - loss: 1.3444 - precision: 0.3620 - recall: 0.2309 - val_accuracy: 0.4060 - val_f1_score: 0.2861 - val_loss: 1.3242 - val_precision: 0.3793 - val_recall: 0.2297
Epoch 35/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3693 - f1_score: 0.2885 - loss: 

Epoch 49/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3974 - f1_score: 0.2738 - loss: 1.2349 - precision: 0.3812 - recall: 0.2137
Epoch 49: saving model to experiment1_ckpts/training_49.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.3975 - f1_score: 0.2740 - loss: 1.2348 - precision: 0.3815 - recall: 0.2138 - val_accuracy: 0.4408 - val_f1_score: 0.3008 - val_loss: 1.2170 - val_precision: 0.4274 - val_recall: 0.2320
Epoch 50/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4051 - f1_score: 0.2974 - loss: 1.2254 - precision: 0.4143 - recall: 0.2319
Epoch 50: saving model to experiment1_ckpts/training_50.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.4051 - f1_score: 0.2974 - loss: 1.2253 - precision: 0.4143 - recall: 0.2319 - val_accuracy: 0.4385 - val_f1_score: 0.3077 - val_loss: 1.2280 - val_precision: 0.4397 - val_recall: 0.2367
Epoch 51/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4100 - f1_score: 0.3017 - loss: 

Epoch 65/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4314 - f1_score: 0.3211 - loss: 1.1387 - precision: 0.4731 - recall: 0.2431
Epoch 65: saving model to experiment1_ckpts/training_65.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.4314 - f1_score: 0.3211 - loss: 1.1387 - precision: 0.4731 - recall: 0.2431 - val_accuracy: 0.4548 - val_f1_score: 0.3040 - val_loss: 1.1414 - val_precision: 0.4897 - val_recall: 0.2204
Epoch 66/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4329 - f1_score: 0.3128 - loss: 1.1367 - precision: 0.4776 - recall: 0.2327
Epoch 66: saving model to experiment1_ckpts/training_66.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.4329 - f1_score: 0.3128 - loss: 1.1366 - precision: 0.4776 - recall: 0.2327 - val_accuracy: 0.4571 - val_f1_score: 0.3067 - val_loss: 1.1375 - val_precision: 0.4923 - val_recall: 0.2227
Epoch 67/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4371 - f1_score: 0.3074 - loss: 

## Experiment 2
8 and 3 neuron layers

In [33]:
checkpoint_path = "experiment2_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
for layer in conv_base.layers:
        layer.trainable = False
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = False
model.layers[-7].trainable = False

/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 1222 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [34]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

Epoch 1/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4026 - f1_score: 0.3208 - loss: 1.2750 - precision: 0.3981 - recall: 0.2687
Epoch 1: saving model to experiment2_ckpts/training_1.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.4025 - f1_score: 0.3207 - loss: 1.2752 - precision: 0.3981 - recall: 0.2687 - val_accuracy: 0.3016 - val_f1_score: 0.0000e+00 - val_loss: 1.1127 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3923 - f1_score: 0.3187 - loss: 1.2831 - precision: 0.3945 - recall: 0.2674
Epoch 2: saving model to experiment2_ckpts/training_2.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.3923 - f1_score: 0.3187 - loss: 1.2831 - precision: 0.3945 - recall: 0.2674 - val_accuracy: 0.3689 - val_f1_score: 0.0356 - val_loss: 1.1127 - val_precision: 0.4211 - val_recall: 0.0186
Epoch 3/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4021 - f1_score: 0.3220 - l

Epoch 17/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4516 - f1_score: 0.3119 - loss: 1.1000 - precision: 0.5255 - recall: 0.2218
Epoch 17: saving model to experiment2_ckpts/training_17.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.4516 - f1_score: 0.3119 - loss: 1.1001 - precision: 0.5256 - recall: 0.2218 - val_accuracy: 0.4200 - val_f1_score: 0.3023 - val_loss: 1.1301 - val_precision: 0.4921 - val_recall: 0.2181
Epoch 18/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4524 - f1_score: 0.3027 - loss: 1.0917 - precision: 0.5267 - recall: 0.2124
Epoch 18: saving model to experiment2_ckpts/training_18.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.4524 - f1_score: 0.3027 - loss: 1.0917 - precision: 0.5267 - recall: 0.2125 - val_accuracy: 0.4269 - val_f1_score: 0.3074 - val_loss: 1.1089 - val_precision: 0.5080 - val_recall: 0.2204
Epoch 19/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4609 - f1_score: 0.3044 - loss: 

Epoch 33/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5180 - f1_score: 0.3002 - loss: 1.0301 - precision: 0.7475 - recall: 0.1878
Epoch 33: saving model to experiment2_ckpts/training_33.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 147s 1s/step - accuracy: 0.5180 - f1_score: 0.3002 - loss: 1.0301 - precision: 0.7474 - recall: 0.1878 - val_accuracy: 0.4780 - val_f1_score: 0.3009 - val_loss: 1.0365 - val_precision: 0.7193 - val_recall: 0.1903
Epoch 34/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5236 - f1_score: 0.2974 - loss: 1.0193 - precision: 0.7293 - recall: 0.1869
Epoch 34: saving model to experiment2_ckpts/training_34.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 147s 1s/step - accuracy: 0.5236 - f1_score: 0.2974 - loss: 1.0193 - precision: 0.7293 - recall: 0.1869 - val_accuracy: 0.4872 - val_f1_score: 0.2972 - val_loss: 1.0418 - val_precision: 0.7105 - val_recall: 0.1879
Epoch 35/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5215 - f1_score: 0.3001 - loss: 

Epoch 49/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5877 - f1_score: 0.3395 - loss: 0.9717 - precision: 0.7989 - recall: 0.2156
Epoch 49: saving model to experiment2_ckpts/training_49.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.5876 - f1_score: 0.3396 - loss: 0.9717 - precision: 0.7989 - recall: 0.2156 - val_accuracy: 0.5499 - val_f1_score: 0.3309 - val_loss: 0.9922 - val_precision: 0.7965 - val_recall: 0.2088
Epoch 50/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5617 - f1_score: 0.3481 - loss: 0.9798 - precision: 0.8061 - recall: 0.2221
Epoch 50: saving model to experiment2_ckpts/training_50.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.5618 - f1_score: 0.3481 - loss: 0.9798 - precision: 0.8061 - recall: 0.2221 - val_accuracy: 0.5499 - val_f1_score: 0.3297 - val_loss: 0.9958 - val_precision: 0.7826 - val_recall: 0.2088
Epoch 51/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5742 - f1_score: 0.3520 - loss: 

Epoch 65/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6122 - f1_score: 0.4133 - loss: 0.9443 - precision: 0.8167 - recall: 0.2768
Epoch 65: saving model to experiment2_ckpts/training_65.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.6122 - f1_score: 0.4134 - loss: 0.9443 - precision: 0.8168 - recall: 0.2768 - val_accuracy: 0.5870 - val_f1_score: 0.4129 - val_loss: 0.9602 - val_precision: 0.7625 - val_recall: 0.2831
Epoch 66/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6086 - f1_score: 0.4128 - loss: 0.9479 - precision: 0.8021 - recall: 0.2780
Epoch 66: saving model to experiment2_ckpts/training_66.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.6086 - f1_score: 0.4129 - loss: 0.9478 - precision: 0.8021 - recall: 0.2780 - val_accuracy: 0.5824 - val_f1_score: 0.4088 - val_loss: 0.9714 - val_precision: 0.7516 - val_recall: 0.2807
Epoch 67/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6170 - f1_score: 0.4255 - loss: 

## Experimento 3
All dense layers

In [38]:
checkpoint_path = "experiment3_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
for layer in conv_base.layers:
        layer.trainable = False
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True
model.layers[-7].trainable = False

/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 1222 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [39]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

Epoch 1/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4522 - f1_score: 0.3863 - loss: 1.6934 - precision: 0.5030 - recall: 0.3143
Epoch 1: saving model to experiment3_ckpts/training_1.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - accuracy: 0.4527 - f1_score: 0.3868 - loss: 1.6925 - precision: 0.5037 - recall: 0.3146 - val_accuracy: 0.4617 - val_f1_score: 0.0536 - val_loss: 1.6073 - val_precision: 0.2407 - val_recall: 0.0302
Epoch 2/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6264 - f1_score: 0.5880 - loss: 1.4568 - precision: 0.7135 - recall: 0.5006
Epoch 2: saving model to experiment3_ckpts/training_2.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.6266 - f1_score: 0.5881 - loss: 1.4569 - precision: 0.7135 - recall: 0.5008 - val_accuracy: 0.6937 - val_f1_score: 0.5148 - val_loss: 1.4605 - val_precision: 0.8771 - val_recall: 0.3643
Epoch 3/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6647 - f1_score: 0.6372 - loss: 1.4825 

Epoch 17/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7830 - f1_score: 0.7667 - loss: 1.1385 - precision: 0.8390 - recall: 0.7061
Epoch 17: saving model to experiment3_ckpts/training_17.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.7829 - f1_score: 0.7666 - loss: 1.1387 - precision: 0.8388 - recall: 0.7059 - val_accuracy: 0.7262 - val_f1_score: 0.7080 - val_loss: 1.2257 - val_precision: 0.7778 - val_recall: 0.6497
Epoch 18/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7750 - f1_score: 0.7647 - loss: 1.1129 - precision: 0.8369 - recall: 0.7040
Epoch 18: saving model to experiment3_ckpts/training_18.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 147s 1s/step - accuracy: 0.7749 - f1_score: 0.7646 - loss: 1.1131 - precision: 0.8368 - recall: 0.7039 - val_accuracy: 0.7309 - val_f1_score: 0.7184 - val_loss: 1.2491 - val_precision: 0.7799 - val_recall: 0.6659
Epoch 19/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7850 - f1_score: 0.7672 - loss: 

Epoch 33/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8162 - f1_score: 0.8046 - loss: 0.9800 - precision: 0.8676 - recall: 0.7502
Epoch 33: saving model to experiment3_ckpts/training_33.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.8162 - f1_score: 0.8045 - loss: 0.9801 - precision: 0.8675 - recall: 0.7501 - val_accuracy: 0.7239 - val_f1_score: 0.7037 - val_loss: 1.1028 - val_precision: 0.7707 - val_recall: 0.6473
Epoch 34/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7985 - f1_score: 0.7953 - loss: 0.9932 - precision: 0.8660 - recall: 0.7353
Epoch 34: saving model to experiment3_ckpts/training_34.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.7985 - f1_score: 0.7953 - loss: 0.9932 - precision: 0.8659 - recall: 0.7353 - val_accuracy: 0.7425 - val_f1_score: 0.7268 - val_loss: 1.1615 - val_precision: 0.7902 - val_recall: 0.6729
Epoch 35/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7987 - f1_score: 0.7832 - loss: 

Epoch 49/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8442 - f1_score: 0.8313 - loss: 0.8754 - precision: 0.8912 - recall: 0.7790
Epoch 49: saving model to experiment3_ckpts/training_49.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - accuracy: 0.8441 - f1_score: 0.8313 - loss: 0.8755 - precision: 0.8912 - recall: 0.7790 - val_accuracy: 0.6984 - val_f1_score: 0.6839 - val_loss: 1.1702 - val_precision: 0.7277 - val_recall: 0.6450
Epoch 50/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8460 - f1_score: 0.8277 - loss: 0.8862 - precision: 0.8935 - recall: 0.7710
Epoch 50: saving model to experiment3_ckpts/training_50.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.8460 - f1_score: 0.8277 - loss: 0.8862 - precision: 0.8934 - recall: 0.7710 - val_accuracy: 0.7633 - val_f1_score: 0.7637 - val_loss: 1.0125 - val_precision: 0.8231 - val_recall: 0.7123
Epoch 51/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8361 - f1_score: 0.8242 - loss: 

Epoch 65/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8695 - f1_score: 0.8640 - loss: 0.8209 - precision: 0.9125 - recall: 0.8204
Epoch 65: saving model to experiment3_ckpts/training_65.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 144s 1s/step - accuracy: 0.8694 - f1_score: 0.8639 - loss: 0.8209 - precision: 0.9125 - recall: 0.8203 - val_accuracy: 0.7842 - val_f1_score: 0.7744 - val_loss: 0.9706 - val_precision: 0.8420 - val_recall: 0.7169
Epoch 66/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8526 - f1_score: 0.8446 - loss: 0.8317 - precision: 0.8970 - recall: 0.7979
Epoch 66: saving model to experiment3_ckpts/training_66.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.8526 - f1_score: 0.8446 - loss: 0.8316 - precision: 0.8970 - recall: 0.7980 - val_accuracy: 0.7587 - val_f1_score: 0.7574 - val_loss: 1.0195 - val_precision: 0.8117 - val_recall: 0.7100
Epoch 67/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8851 - f1_score: 0.8789 - loss: 

# Experiment 4
Copy to mammography training

In [43]:
checkpoint_path = "experiment4_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 1222 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [44]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

Epoch 1/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.5252 - f1_score: 0.4968 - loss: 1.6969 - precision: 0.5705 - recall: 0.4402
Epoch 1: saving model to experiment4_ckpts/training_1.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 912s 7s/step - accuracy: 0.5258 - f1_score: 0.4975 - loss: 1.6963 - precision: 0.5711 - recall: 0.4408 - val_accuracy: 0.5104 - val_f1_score: 0.1988 - val_loss: 1.6875 - val_precision: 0.7903 - val_recall: 0.1137
Epoch 2/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.7727 - f1_score: 0.7571 - loss: 1.3408 - precision: 0.8244 - recall: 0.7000
Epoch 2: saving model to experiment4_ckpts/training_2.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 852s 7s/step - accuracy: 0.7725 - f1_score: 0.7570 - loss: 1.3410 - precision: 0.8242 - recall: 0.6999 - val_accuracy: 0.6566 - val_f1_score: 0.5942 - val_loss: 1.4817 - val_precision: 0.7297 - val_recall: 0.5012
Epoch 3/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.8172 - f1_score: 0.8122 - loss: 1.1910 

Epoch 17/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9673 - f1_score: 0.9653 - loss: 0.5951 - precision: 0.9710 - recall: 0.9597
Epoch 17: saving model to experiment4_ckpts/training_17.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 842s 7s/step - accuracy: 0.9673 - f1_score: 0.9654 - loss: 0.5950 - precision: 0.9711 - recall: 0.9598 - val_accuracy: 0.7819 - val_f1_score: 0.7914 - val_loss: 0.9797 - val_precision: 0.8189 - val_recall: 0.7657
Epoch 18/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9798 - f1_score: 0.9790 - loss: 0.5620 - precision: 0.9828 - recall: 0.9753
Epoch 18: saving model to experiment4_ckpts/training_18.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 832s 7s/step - accuracy: 0.9798 - f1_score: 0.9790 - loss: 0.5620 - precision: 0.9828 - recall: 0.9753 - val_accuracy: 0.8005 - val_f1_score: 0.7981 - val_loss: 0.9172 - val_precision: 0.8175 - val_recall: 0.7796
Epoch 19/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9815 - f1_score: 0.9807 - loss: 

Epoch 33/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9893 - f1_score: 0.9892 - loss: 0.4688 - precision: 0.9903 - recall: 0.9880
Epoch 33: saving model to experiment4_ckpts/training_33.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 784s 6s/step - accuracy: 0.9893 - f1_score: 0.9892 - loss: 0.4687 - precision: 0.9903 - recall: 0.9880 - val_accuracy: 0.7935 - val_f1_score: 0.7986 - val_loss: 0.8778 - val_precision: 0.8160 - val_recall: 0.7819
Epoch 34/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9856 - f1_score: 0.9858 - loss: 0.4840 - precision: 0.9886 - recall: 0.9829
Epoch 34: saving model to experiment4_ckpts/training_34.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 810s 7s/step - accuracy: 0.9856 - f1_score: 0.9857 - loss: 0.4839 - precision: 0.9886 - recall: 0.9829 - val_accuracy: 0.7981 - val_f1_score: 0.8009 - val_loss: 0.8398 - val_precision: 0.8184 - val_recall: 0.7842
Epoch 35/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9881 - f1_score: 0.9882 - loss: 

Epoch 49/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9896 - f1_score: 0.9897 - loss: 0.4360 - precision: 0.9911 - recall: 0.9883
Epoch 49: saving model to experiment4_ckpts/training_49.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 821s 7s/step - accuracy: 0.9896 - f1_score: 0.9897 - loss: 0.4361 - precision: 0.9911 - recall: 0.9883 - val_accuracy: 0.8190 - val_f1_score: 0.8208 - val_loss: 0.7723 - val_precision: 0.8345 - val_recall: 0.8074
Epoch 50/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9903 - f1_score: 0.9898 - loss: 0.4449 - precision: 0.9911 - recall: 0.9885
Epoch 50: saving model to experiment4_ckpts/training_50.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 801s 7s/step - accuracy: 0.9903 - f1_score: 0.9897 - loss: 0.4450 - precision: 0.9910 - recall: 0.9884 - val_accuracy: 0.8167 - val_f1_score: 0.8202 - val_loss: 0.8059 - val_precision: 0.8310 - val_recall: 0.8097
Epoch 51/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9939 - f1_score: 0.9939 - loss: 

COMET ERROR: Heartbeat processing error


122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9891 - f1_score: 0.9891 - loss: 0.4374 - precision: 0.9895 - recall: 0.9887
Epoch 57: saving model to experiment4_ckpts/training_57.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 849s 7s/step - accuracy: 0.9891 - f1_score: 0.9891 - loss: 0.4375 - precision: 0.9895 - recall: 0.9886 - val_accuracy: 0.8121 - val_f1_score: 0.8152 - val_loss: 0.8510 - val_precision: 0.8329 - val_recall: 0.7981
Epoch 58/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9897 - f1_score: 0.9899 - loss: 0.4480 - precision: 0.9911 - recall: 0.9886
Epoch 58: saving model to experiment4_ckpts/training_58.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 846s 7s/step - accuracy: 0.9897 - f1_score: 0.9899 - loss: 0.4480 - precision: 0.9911 - recall: 0.9886 - val_accuracy: 0.8144 - val_f1_score: 0.8123 - val_loss: 0.7941 - val_precision: 0.8269 - val_recall: 0.7981
Epoch 59/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9915 - f1_score: 0.9911 - loss: 0.4342 - pre

# Experiment 5
ResNet less defreeze

In [60]:
checkpoint_path = "experiment5_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = ResNet50(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:3]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)

model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [61]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

Epoch 1/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4061 - f1_score: 0.3684 - loss: 793.1513 - precision: 0.4448 - recall: 0.3144
Epoch 1: saving model to experiment5_ckpts/training_1.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 450s 4s/step - accuracy: 0.4064 - f1_score: 0.3687 - loss: 793.0615 - precision: 0.4452 - recall: 0.3146 - val_accuracy: 0.2715 - val_f1_score: 0.0000e+00 - val_loss: 759.7882 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5874 - f1_score: 0.5518 - loss: 748.7540 - precision: 0.6652 - recall: 0.4715
Epoch 2: saving model to experiment5_ckpts/training_2.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 424s 3s/step - accuracy: 0.5875 - f1_score: 0.5519 - loss: 748.6656 - precision: 0.6652 - recall: 0.4717 - val_accuracy: 0.2715 - val_f1_score: 0.0000e+00 - val_loss: 716.0321 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.668

Epoch 17/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9480 - f1_score: 0.9419 - loss: 216.2251 - precision: 0.9659 - recall: 0.9191
Epoch 17: saving model to experiment5_ckpts/training_17.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 420s 3s/step - accuracy: 0.9479 - f1_score: 0.9419 - loss: 216.1756 - precision: 0.9659 - recall: 0.9191 - val_accuracy: 0.7146 - val_f1_score: 0.7086 - val_loss: 198.2771 - val_precision: 0.7573 - val_recall: 0.6659
Epoch 18/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9484 - f1_score: 0.9447 - loss: 192.0537 - precision: 0.9647 - recall: 0.9254
Epoch 18: saving model to experiment5_ckpts/training_18.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 430s 4s/step - accuracy: 0.9484 - f1_score: 0.9446 - loss: 192.0072 - precision: 0.9647 - recall: 0.9254 - val_accuracy: 0.7100 - val_f1_score: 0.7022 - val_loss: 175.2153 - val_precision: 0.7547 - val_recall: 0.6566
Epoch 19/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9438 - f1_score: 0.9

Epoch 33/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9627 - f1_score: 0.9628 - loss: 8.0231 - precision: 0.9704 - recall: 0.9553
Epoch 33: saving model to experiment5_ckpts/training_33.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 429s 4s/step - accuracy: 0.9627 - f1_score: 0.9628 - loss: 8.0183 - precision: 0.9704 - recall: 0.9553 - val_accuracy: 0.7262 - val_f1_score: 0.7183 - val_loss: 6.8089 - val_precision: 0.7500 - val_recall: 0.6891
Epoch 34/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9615 - f1_score: 0.9608 - loss: 5.8978 - precision: 0.9672 - recall: 0.9545
Epoch 34: saving model to experiment5_ckpts/training_34.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 434s 4s/step - accuracy: 0.9615 - f1_score: 0.9608 - loss: 5.8943 - precision: 0.9672 - recall: 0.9544 - val_accuracy: 0.7332 - val_f1_score: 0.7255 - val_loss: 5.1416 - val_precision: 0.7469 - val_recall: 0.7053
Epoch 35/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9464 - f1_score: 0.9426 - loss: 

Epoch 49/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9770 - f1_score: 0.9769 - loss: 0.6196 - precision: 0.9799 - recall: 0.9738
Epoch 49: saving model to experiment5_ckpts/training_49.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 420s 3s/step - accuracy: 0.9770 - f1_score: 0.9769 - loss: 0.6195 - precision: 0.9799 - recall: 0.9738 - val_accuracy: 0.7865 - val_f1_score: 0.7795 - val_loss: 0.9479 - val_precision: 0.8015 - val_recall: 0.7587
Epoch 50/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9844 - f1_score: 0.9839 - loss: 0.5553 - precision: 0.9861 - recall: 0.9816
Epoch 50: saving model to experiment5_ckpts/training_50.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 420s 3s/step - accuracy: 0.9844 - f1_score: 0.9839 - loss: 0.5553 - precision: 0.9861 - recall: 0.9816 - val_accuracy: 0.7796 - val_f1_score: 0.7665 - val_loss: 0.9666 - val_precision: 0.7921 - val_recall: 0.7425
Epoch 51/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9838 - f1_score: 0.9838 - loss: 

Epoch 65/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9859 - f1_score: 0.9867 - loss: 0.4672 - precision: 0.9906 - recall: 0.9828
Epoch 65: saving model to experiment5_ckpts/training_65.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 427s 3s/step - accuracy: 0.9859 - f1_score: 0.9867 - loss: 0.4673 - precision: 0.9906 - recall: 0.9828 - val_accuracy: 0.7958 - val_f1_score: 0.7991 - val_loss: 0.9074 - val_precision: 0.8145 - val_recall: 0.7842
Epoch 66/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9856 - f1_score: 0.9861 - loss: 0.4883 - precision: 0.9882 - recall: 0.9840
Epoch 66: saving model to experiment5_ckpts/training_66.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 425s 3s/step - accuracy: 0.9856 - f1_score: 0.9861 - loss: 0.4882 - precision: 0.9882 - recall: 0.9839 - val_accuracy: 0.7796 - val_f1_score: 0.7761 - val_loss: 0.9743 - val_precision: 0.7844 - val_recall: 0.7680
Epoch 67/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9845 - f1_score: 0.9842 - loss: 

# Experiment 6
ResNet

In [52]:
checkpoint_path = "experiment6_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = ResNet50(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)

model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [53]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

Epoch 1/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.3413 - f1_score: 0.2365 - loss: 791.9819 - precision: 0.3195 - recall: 0.1877
Epoch 1: saving model to experiment6_ckpts/training_1.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 486s 4s/step - accuracy: 0.3416 - f1_score: 0.2367 - loss: 791.8842 - precision: 0.3198 - recall: 0.1879 - val_accuracy: 0.5940 - val_f1_score: 0.0000e+00 - val_loss: 755.5490 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5253 - f1_score: 0.4158 - loss: 743.8797 - precision: 0.5677 - recall: 0.3280
Epoch 2: saving model to experiment6_ckpts/training_2.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 448s 4s/step - accuracy: 0.5254 - f1_score: 0.4159 - loss: 743.7850 - precision: 0.5678 - recall: 0.3281 - val_accuracy: 0.5940 - val_f1_score: 0.5940 - val_loss: 708.6888 - val_precision: 0.5940 - val_recall: 0.5940
Epoch 3/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6130 - f1_score

Epoch 17/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9288 - f1_score: 0.8971 - loss: 201.3139 - precision: 0.9589 - recall: 0.8431
Epoch 17: saving model to experiment6_ckpts/training_17.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 409s 3s/step - accuracy: 0.9288 - f1_score: 0.8971 - loss: 201.2649 - precision: 0.9589 - recall: 0.8432 - val_accuracy: 0.7007 - val_f1_score: 0.6491 - val_loss: 183.5446 - val_precision: 0.7523 - val_recall: 0.5708
Epoch 18/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9464 - f1_score: 0.9220 - loss: 177.5422 - precision: 0.9710 - recall: 0.8778
Epoch 18: saving model to experiment6_ckpts/training_18.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 409s 3s/step - accuracy: 0.9463 - f1_score: 0.9219 - loss: 177.4966 - precision: 0.9709 - recall: 0.8777 - val_accuracy: 0.6914 - val_f1_score: 0.6445 - val_loss: 161.0017 - val_precision: 0.7563 - val_recall: 0.5615
Epoch 19/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9344 - f1_score: 0.9

Epoch 33/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9458 - f1_score: 0.9370 - loss: 5.4389 - precision: 0.9598 - recall: 0.9153
Epoch 33: saving model to experiment6_ckpts/training_33.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 425s 3s/step - accuracy: 0.9458 - f1_score: 0.9371 - loss: 5.4351 - precision: 0.9598 - recall: 0.9154 - val_accuracy: 0.7541 - val_f1_score: 0.7404 - val_loss: 4.4387 - val_precision: 0.7968 - val_recall: 0.6914
Epoch 34/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9539 - f1_score: 0.9497 - loss: 3.7982 - precision: 0.9645 - recall: 0.9353
Epoch 34: saving model to experiment6_ckpts/training_34.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 417s 3s/step - accuracy: 0.9539 - f1_score: 0.9497 - loss: 3.7958 - precision: 0.9645 - recall: 0.9353 - val_accuracy: 0.7239 - val_f1_score: 0.7150 - val_loss: 3.2739 - val_precision: 0.7751 - val_recall: 0.6636
Epoch 35/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9599 - f1_score: 0.9555 - loss: 

Epoch 49/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9799 - f1_score: 0.9796 - loss: 0.5685 - precision: 0.9839 - recall: 0.9754
Epoch 49: saving model to experiment6_ckpts/training_49.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 410s 3s/step - accuracy: 0.9800 - f1_score: 0.9796 - loss: 0.5684 - precision: 0.9839 - recall: 0.9754 - val_accuracy: 0.7749 - val_f1_score: 0.7604 - val_loss: 0.8800 - val_precision: 0.8036 - val_recall: 0.7216
Epoch 50/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9877 - f1_score: 0.9869 - loss: 0.5289 - precision: 0.9913 - recall: 0.9826
Epoch 50: saving model to experiment6_ckpts/training_50.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 413s 3s/step - accuracy: 0.9876 - f1_score: 0.9869 - loss: 0.5290 - precision: 0.9913 - recall: 0.9825 - val_accuracy: 0.7726 - val_f1_score: 0.7747 - val_loss: 0.8898 - val_precision: 0.8154 - val_recall: 0.7378
Epoch 51/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9888 - f1_score: 0.9857 - loss: 

Epoch 65/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9859 - f1_score: 0.9870 - loss: 0.4956 - precision: 0.9900 - recall: 0.9840
Epoch 65: saving model to experiment6_ckpts/training_65.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 405s 3s/step - accuracy: 0.9859 - f1_score: 0.9870 - loss: 0.4955 - precision: 0.9900 - recall: 0.9840 - val_accuracy: 0.7865 - val_f1_score: 0.7913 - val_loss: 0.8388 - val_precision: 0.8241 - val_recall: 0.7610
Epoch 66/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9897 - f1_score: 0.9894 - loss: 0.4817 - precision: 0.9927 - recall: 0.9861
Epoch 66: saving model to experiment6_ckpts/training_66.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 405s 3s/step - accuracy: 0.9897 - f1_score: 0.9894 - loss: 0.4817 - precision: 0.9927 - recall: 0.9861 - val_accuracy: 0.7981 - val_f1_score: 0.7889 - val_loss: 0.8337 - val_precision: 0.8216 - val_recall: 0.7587
Epoch 67/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9908 - f1_score: 0.9901 - loss: 

# Experiment 7
Inception

In [69]:
checkpoint_path = "experiment7_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = InceptionV3(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)

model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

In [70]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

Epoch 1/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 267s 2s/step - accuracy: 0.3335 - f1_score: 0.2481 - loss: 791.9163 - precision: 0.3209 - recall: 0.2022 - val_accuracy: 0.4524 - val_f1_score: 0.0138 - val_loss: 756.4088 - val_precision: 1.0000 - val_recall: 0.0070
Epoch 2/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 245s 2s/step - accuracy: 0.4555 - f1_score: 0.3612 - loss: 744.8758 - precision: 0.4719 - recall: 0.2926 - val_accuracy: 0.4200 - val_f1_score: 0.0748 - val_loss: 710.4543 - val_precision: 0.3600 - val_recall: 0.0418
Epoch 3/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 247s 2s/step - accuracy: 0.5425 - f1_score: 0.4627 - loss: 699.0817 - precision: 0.5770 - recall: 0.3862 - val_accuracy: 0.4432 - val_f1_score: 0.2367 - val_loss: 665.6190 - val_precision: 0.4201 - val_recall: 0.1647
Epoch 4/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 247s 2s/step - accuracy: 0.6101 - f1_score: 0.5465 - loss: 654.4755 - precision: 0.6687 - recall: 0.4621 - val_accuracy: 0.4803 - val_f1_score: 0.3797 - val_loss: 622.0167 - val_precision: 

122/122 ━━━━━━━━━━━━━━━━━━━━ 255s 2s/step - accuracy: 0.9871 - f1_score: 0.9860 - loss: 0.5009 - precision: 0.9887 - recall: 0.9834 - val_accuracy: 0.7865 - val_f1_score: 0.7872 - val_loss: 0.8742 - val_precision: 0.8024 - val_recall: 0.7726
Epoch 58/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.9827 - f1_score: 0.9802 - loss: 0.5142 - precision: 0.9834 - recall: 0.9770 - val_accuracy: 0.7912 - val_f1_score: 0.7867 - val_loss: 0.8372 - val_precision: 0.8088 - val_recall: 0.7657
Epoch 59/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 249s 2s/step - accuracy: 0.9814 - f1_score: 0.9815 - loss: 0.5059 - precision: 0.9848 - recall: 0.9783 - val_accuracy: 0.7796 - val_f1_score: 0.7900 - val_loss: 0.8594 - val_precision: 0.8133 - val_recall: 0.7680
Epoch 60/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 249s 2s/step - accuracy: 0.9795 - f1_score: 0.9784 - loss: 0.5092 - precision: 0.9832 - recall: 0.9736 - val_accuracy: 0.7865 - val_f1_score: 0.7891 - val_loss: 0.8407 - val_precision: 0.8063 - val_recall: 0.7

# Experiment 8
Inception more lr

In [74]:
checkpoint_path = "experiment7_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = InceptionV3(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/phantoms_training_experiment7_seeded.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 402 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [75]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

Epoch 1/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 274s 2s/step - accuracy: 0.6388 - f1_score: 0.6152 - loss: 0.9161 - precision: 0.9436 - recall: 0.4910 - val_accuracy: 0.7842 - val_f1_score: 0.0000e+00 - val_loss: 1.0854 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 255s 2s/step - accuracy: 0.9891 - f1_score: 0.9736 - loss: 0.5792 - precision: 0.9953 - recall: 0.9529 - val_accuracy: 0.7796 - val_f1_score: 0.0046 - val_loss: 1.0246 - val_precision: 1.0000 - val_recall: 0.0023
Epoch 3/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 257s 2s/step - accuracy: 0.9881 - f1_score: 0.9743 - loss: 0.5507 - precision: 0.9930 - recall: 0.9563 - val_accuracy: 0.7842 - val_f1_score: 0.3071 - val_loss: 0.9438 - val_precision: 0.8889 - val_recall: 0.1856
Epoch 4/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 265s 2s/step - accuracy: 0.9871 - f1_score: 0.9805 - loss: 0.5408 - precision: 0.9934 - recall: 0.9680 - val_accuracy: 0.7796 - val_f1_score: 0.6522 - val_loss: 0.8813 - val_precision: 0.86

122/122 ━━━━━━━━━━━━━━━━━━━━ 265s 2s/step - accuracy: 0.9890 - f1_score: 0.9896 - loss: 0.5017 - precision: 0.9934 - recall: 0.9860 - val_accuracy: 0.7842 - val_f1_score: 0.7854 - val_loss: 0.7667 - val_precision: 0.8278 - val_recall: 0.7471
Epoch 58/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 247s 2s/step - accuracy: 0.9892 - f1_score: 0.9873 - loss: 0.5075 - precision: 0.9939 - recall: 0.9808 - val_accuracy: 0.7842 - val_f1_score: 0.7854 - val_loss: 0.7618 - val_precision: 0.8278 - val_recall: 0.7471
Epoch 59/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.9923 - f1_score: 0.9901 - loss: 0.4871 - precision: 0.9932 - recall: 0.9871 - val_accuracy: 0.7773 - val_f1_score: 0.7830 - val_loss: 0.7755 - val_precision: 0.8198 - val_recall: 0.7494
Epoch 60/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.9899 - f1_score: 0.9879 - loss: 0.4935 - precision: 0.9946 - recall: 0.9812 - val_accuracy: 0.7773 - val_f1_score: 0.7844 - val_loss: 0.7758 - val_precision: 0.8256 - val_recall: 0.7

# Experiment 9
Change learning rate

In [109]:
checkpoint_path = "experiment4_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 1222 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [110]:
model.fit(
    train_generator,
    #steps_per_epoch=STEP_SIZE_TRAIN,
    epochs=70,
    validation_data=valid_generator,
    #validation_steps=STEP_SIZE_VALID,
    #callbacks=[my_callbacks]
)

Epoch 1/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 924s 7s/step - accuracy: 0.5786 - f1_score: 0.5423 - loss: 1.5587 - precision: 0.6253 - recall: 0.4791 - val_accuracy: 0.5290 - val_f1_score: 0.2087 - val_loss: 1.6443 - val_precision: 0.5729 - val_recall: 0.1276
Epoch 2/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 875s 7s/step - accuracy: 0.7452 - f1_score: 0.7309 - loss: 1.2491 - precision: 0.7860 - recall: 0.6830 - val_accuracy: 0.7285 - val_f1_score: 0.6412 - val_loss: 1.3468 - val_precision: 0.8195 - val_recall: 0.5267
Epoch 3/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 878s 7s/step - accuracy: 0.8079 - f1_score: 0.8022 - loss: 1.0866 - precision: 0.8405 - recall: 0.7673 - val_accuracy: 0.7309 - val_f1_score: 0.7216 - val_loss: 1.2516 - val_precision: 0.7811 - val_recall: 0.6705
Epoch 4/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 929s 8s/step - accuracy: 0.8412 - f1_score: 0.8357 - loss: 0.9731 - precision: 0.8589 - recall: 0.8138 - val_accuracy: 0.7494 - val_f1_score: 0.7358 - val_loss: 1.1687 - val_precision: 0.7863 - val_rec

122/122 ━━━━━━━━━━━━━━━━━━━━ 848s 7s/step - accuracy: 0.9943 - f1_score: 0.9941 - loss: 0.1788 - precision: 0.9943 - recall: 0.9939 - val_accuracy: 0.8167 - val_f1_score: 0.8182 - val_loss: 0.6517 - val_precision: 0.8220 - val_recall: 0.8144
Epoch 58/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 858s 7s/step - accuracy: 0.9933 - f1_score: 0.9936 - loss: 0.1841 - precision: 0.9942 - recall: 0.9930 - val_accuracy: 0.8005 - val_f1_score: 0.8080 - val_loss: 0.6673 - val_precision: 0.8156 - val_recall: 0.8005
Epoch 59/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 862s 7s/step - accuracy: 0.9923 - f1_score: 0.9914 - loss: 0.1953 - precision: 0.9923 - recall: 0.9905 - val_accuracy: 0.8121 - val_f1_score: 0.8145 - val_loss: 0.6848 - val_precision: 0.8192 - val_recall: 0.8097
Epoch 60/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 867s 7s/step - accuracy: 0.9909 - f1_score: 0.9907 - loss: 0.1974 - precision: 0.9911 - recall: 0.9902 - val_accuracy: 0.8167 - val_f1_score: 0.8173 - val_loss: 0.6788 - val_precision: 0.8251 - val_recall: 0.8

# Experiment 10
Data augmentation

In [23]:
datagen_aug=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess,vertical_flip=True)
train_generator_aug=datagen_aug.flow_from_dataframe(
dataframe=X_train,
directory="content/phantom_images/train",
x_col="filename",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

Found 3873 validated image filenames belonging to 3 classes.


In [24]:
checkpoint_path = "experiment4_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 1222 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [25]:
model.fit(
    train_generator_aug,
    epochs=70,
    validation_data=valid_generator,
)

Epoch 1/70


/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


122/122 ━━━━━━━━━━━━━━━━━━━━ 857s 7s/step - accuracy: 0.5895 - f1_score: 0.5538 - loss: 1.6532 - precision: 0.6629 - recall: 0.4757 - val_accuracy: 0.5615 - val_f1_score: 0.2382 - val_loss: 1.6632 - val_precision: 0.6429 - val_recall: 0.1462
Epoch 2/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 833s 7s/step - accuracy: 0.7239 - f1_score: 0.7075 - loss: 1.3889 - precision: 0.7919 - recall: 0.6395 - val_accuracy: 0.7587 - val_f1_score: 0.5469 - val_loss: 1.3584 - val_precision: 0.8687 - val_recall: 0.3991
Epoch 3/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 823s 7s/step - accuracy: 0.7711 - f1_score: 0.7616 - loss: 1.2475 - precision: 0.8317 - recall: 0.7025 - val_accuracy: 0.7541 - val_f1_score: 0.7314 - val_loss: 1.2914 - val_precision: 0.8567 - val_recall: 0.6381
Epoch 4/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 837s 7s/step - accuracy: 0.7821 - f1_score: 0.7705 - loss: 1.2078 - precision: 0.8239 - recall: 0.7236 - val_accuracy: 0.7633 - val_f1_score: 0.7477 - val_loss: 1.1517 - val_precision: 0.8450 - val_recall: 0.6705

122/122 ━━━━━━━━━━━━━━━━━━━━ 715s 6s/step - accuracy: 0.9863 - f1_score: 0.9861 - loss: 0.4609 - precision: 0.9871 - recall: 0.9850 - val_accuracy: 0.8213 - val_f1_score: 0.8175 - val_loss: 0.7848 - val_precision: 0.8354 - val_recall: 0.8005
Epoch 58/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 721s 6s/step - accuracy: 0.9879 - f1_score: 0.9884 - loss: 0.4478 - precision: 0.9892 - recall: 0.9877 - val_accuracy: 0.8306 - val_f1_score: 0.8284 - val_loss: 0.7630 - val_precision: 0.8454 - val_recall: 0.8121
Epoch 59/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 742s 6s/step - accuracy: 0.9921 - f1_score: 0.9919 - loss: 0.4370 - precision: 0.9926 - recall: 0.9912 - val_accuracy: 0.8329 - val_f1_score: 0.8385 - val_loss: 0.7569 - val_precision: 0.8589 - val_recall: 0.8190
Epoch 60/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 731s 6s/step - accuracy: 0.9859 - f1_score: 0.9860 - loss: 0.4792 - precision: 0.9874 - recall: 0.9846 - val_accuracy: 0.8329 - val_f1_score: 0.8331 - val_loss: 0.7859 - val_precision: 0.8502 - val_recall: 0.8

# Experiment 11
More weight decay to the best model

In [11]:
checkpoint_path = "experiment4_ckpts/training_{epoch}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 1222 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [12]:
model.fit(
    train_generator,
    epochs=70,
    validation_data=valid_generator,
)

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/70


/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


122/122 ━━━━━━━━━━━━━━━━━━━━ 885s 7s/step - accuracy: 0.5552 - f1_score: 0.5373 - loss: 1.7459 - precision: 0.5980 - recall: 0.4878 - val_accuracy: 0.4965 - val_f1_score: 0.4318 - val_loss: 1.8078 - val_precision: 0.5568 - val_recall: 0.3527
Epoch 2/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 788s 6s/step - accuracy: 0.7447 - f1_score: 0.7308 - loss: 1.3623 - precision: 0.7850 - recall: 0.6837 - val_accuracy: 0.6218 - val_f1_score: 0.5966 - val_loss: 1.5303 - val_precision: 0.6879 - val_recall: 0.5267
Epoch 3/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 766s 6s/step - accuracy: 0.7969 - f1_score: 0.7918 - loss: 1.1917 - precision: 0.8326 - recall: 0.7549 - val_accuracy: 0.7541 - val_f1_score: 0.7418 - val_loss: 1.2645 - val_precision: 0.8162 - val_recall: 0.6798
Epoch 4/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 774s 6s/step - accuracy: 0.8486 - f1_score: 0.8469 - loss: 1.0794 - precision: 0.8788 - recall: 0.8173 - val_accuracy: 0.7633 - val_f1_score: 0.7596 - val_loss: 1.2131 - val_precision: 0.8105 - val_recall: 0.7146

122/122 ━━━━━━━━━━━━━━━━━━━━ 716s 6s/step - accuracy: 0.9890 - f1_score: 0.9892 - loss: 0.4471 - precision: 0.9899 - recall: 0.9885 - val_accuracy: 0.8399 - val_f1_score: 0.8288 - val_loss: 0.8005 - val_precision: 0.8438 - val_recall: 0.8144
Epoch 58/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 722s 6s/step - accuracy: 0.9902 - f1_score: 0.9901 - loss: 0.4332 - precision: 0.9904 - recall: 0.9899 - val_accuracy: 0.8260 - val_f1_score: 0.8269 - val_loss: 0.7971 - val_precision: 0.8397 - val_recall: 0.8144
Epoch 59/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 731s 6s/step - accuracy: 0.9908 - f1_score: 0.9911 - loss: 0.4448 - precision: 0.9916 - recall: 0.9905 - val_accuracy: 0.8167 - val_f1_score: 0.8249 - val_loss: 0.7532 - val_precision: 0.8536 - val_recall: 0.7981
Epoch 60/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 735s 6s/step - accuracy: 0.9852 - f1_score: 0.9857 - loss: 0.4396 - precision: 0.9872 - recall: 0.9843 - val_accuracy: 0.8097 - val_f1_score: 0.8152 - val_loss: 0.8423 - val_precision: 0.8329 - val_recall: 0.7

# Experiment 12
More augmentation

In [14]:
datagen_aug=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess,
                               vertical_flip=True, 
                               horizontal_flip = True,
                              zoom_range = 0.2)
train_generator_aug=datagen_aug.flow_from_dataframe(
dataframe=X_train,
directory="content/phantom_images/train",
x_col="filename",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

Found 3873 validated image filenames belonging to 3 classes.


In [15]:
checkpoint_path = "training_0/exp12_best.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
               ModelCheckpoint(checkpoint_path, verbose = 1, monitor = "val_accuracy",save_best_only = True, save_weights_only = True),
]
model = Sequential()

conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
    if layer in train_layers:
        layer.trainable = True
model.build((None, 224, 224, 3))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, weight_decay=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score()
    ]
)
model.load_weights('training_0/final_training.weights.h5')
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.layers[-1].trainable = True
model.layers[-3].trainable = True
model.layers[-5].trainable = True

/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 1222 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [16]:
model.fit(
    train_generator_aug,
    epochs=70,
    validation_data=valid_generator,
    callbacks=[my_callbacks]
)

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/70


/Users/julietaumpierrez/Desktop/breast_cancer_detection/.venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.5182 - f1_score: 0.4829 - loss: 1.7564 - precision: 0.5510 - recall: 0.4297
Epoch 1: val_accuracy improved from -inf to 0.48724, saving model to training_0/exp12_best.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 873s 7s/step - accuracy: 0.5187 - f1_score: 0.4834 - loss: 1.7557 - precision: 0.5516 - recall: 0.4302 - val_accuracy: 0.4872 - val_f1_score: 0.4007 - val_loss: 1.6520 - val_precision: 0.6994 - val_recall: 0.2807
Epoch 2/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.6578 - f1_score: 0.6345 - loss: 1.5140 - precision: 0.6889 - recall: 0.5881
Epoch 2: val_accuracy improved from 0.48724 to 0.62413, saving model to training_0/exp12_best.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 830s 7s/step - accuracy: 0.6578 - f1_score: 0.6345 - loss: 1.5140 - precision: 0.6889 - recall: 0.5881 - val_accuracy: 0.6241 - val_f1_score: 0.5668 - val_loss: 1.4756 - val_precision: 0.7860 - val_recall: 0.4432
Epoch 3/70
122/122 ━━━━━━━━━━━━━━━━━━━

Epoch 17/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8247 - f1_score: 0.8254 - loss: 0.9066 - precision: 0.8479 - recall: 0.8042
Epoch 17: val_accuracy improved from 0.83295 to 0.84455, saving model to training_0/exp12_best.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 806s 7s/step - accuracy: 0.8248 - f1_score: 0.8255 - loss: 0.9064 - precision: 0.8480 - recall: 0.8042 - val_accuracy: 0.8445 - val_f1_score: 0.8394 - val_loss: 0.9353 - val_precision: 0.8483 - val_recall: 0.8306
Epoch 18/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.8495 - f1_score: 0.8425 - loss: 0.8630 - precision: 0.8642 - recall: 0.8219
Epoch 18: val_accuracy improved from 0.84455 to 0.84687, saving model to training_0/exp12_best.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 822s 7s/step - accuracy: 0.8494 - f1_score: 0.8425 - loss: 0.8631 - precision: 0.8642 - recall: 0.8218 - val_accuracy: 0.8469 - val_f1_score: 0.8387 - val_loss: 0.9158 - val_precision: 0.8645 - val_recall: 0.8144
Epoch 19/70
122/122 

Epoch 33/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8793 - f1_score: 0.8759 - loss: 0.7035 - precision: 0.8904 - recall: 0.8619
Epoch 33: val_accuracy improved from 0.84919 to 0.85383, saving model to training_0/exp12_best.weights.h5
122/122 ━━━━━━━━━━━━━━━━━━━━ 787s 6s/step - accuracy: 0.8793 - f1_score: 0.8759 - loss: 0.7037 - precision: 0.8904 - recall: 0.8620 - val_accuracy: 0.8538 - val_f1_score: 0.8528 - val_loss: 0.7666 - val_precision: 0.8588 - val_recall: 0.8469
Epoch 34/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8803 - f1_score: 0.8772 - loss: 0.7011 - precision: 0.8914 - recall: 0.8635
Epoch 34: val_accuracy did not improve from 0.85383
122/122 ━━━━━━━━━━━━━━━━━━━━ 770s 6s/step - accuracy: 0.8803 - f1_score: 0.8773 - loss: 0.7011 - precision: 0.8914 - recall: 0.8636 - val_accuracy: 0.8469 - val_f1_score: 0.8433 - val_loss: 0.7652 - val_precision: 0.8565 - val_recall: 0.8306
Epoch 35/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8740 - f

122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9347 - f1_score: 0.9334 - loss: 0.5491 - precision: 0.9383 - recall: 0.9286
Epoch 65: val_accuracy did not improve from 0.86775
122/122 ━━━━━━━━━━━━━━━━━━━━ 785s 6s/step - accuracy: 0.9347 - f1_score: 0.9334 - loss: 0.5492 - precision: 0.9382 - recall: 0.9286 - val_accuracy: 0.8445 - val_f1_score: 0.8468 - val_loss: 0.7901 - val_precision: 0.8538 - val_recall: 0.8399
Epoch 66/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9208 - f1_score: 0.9210 - loss: 0.5716 - precision: 0.9294 - recall: 0.9128
Epoch 66: val_accuracy did not improve from 0.86775
122/122 ━━━━━━━━━━━━━━━━━━━━ 787s 6s/step - accuracy: 0.9208 - f1_score: 0.9210 - loss: 0.5715 - precision: 0.9294 - recall: 0.9129 - val_accuracy: 0.8376 - val_f1_score: 0.8384 - val_loss: 0.7532 - val_precision: 0.8463 - val_recall: 0.8306
Epoch 67/70
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9244 - f1_score: 0.9241 - loss: 0.5607 - precision: 0.9317 - recall: 0.916